# Electra experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import ElectraModel, ElectraConfig, ElectraTokenizer, ElectraForSequenceClassification
from transformers import Trainer, TrainingArguments


In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_5.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Electra

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro', zero_division=0)
    precision = precision_score(y_true=labels, y_pred=pred, average='macro', zero_division=0)
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

81


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.151208,0.127660,0.002240,0.017544,0.003972
2,No log,3.924424,0.170213,0.017908,0.029240,0.018092
3,No log,3.744459,0.148936,0.014115,0.023392,0.013477
4,4.129500,3.538238,0.191489,0.011444,0.035088,0.017052
5,4.129500,3.392758,0.198582,0.013708,0.038793,0.019995
6,4.129500,3.218445,0.283688,0.050760,0.075963,0.057920
7,3.576700,3.100243,0.312057,0.053550,0.083128,0.062929
8,3.576700,3.006285,0.319149,0.054020,0.096247,0.065915
9,3.576700,2.961973,0.326241,0.055155,0.100484,0.067880
10,3.087900,2.893004,0.368794,0.087258,0.135997,0.091298


TrainOutput(global_step=2385, training_loss=3.266777502415815, metrics={'train_runtime': 263.5997, 'train_samples_per_second': 72.098, 'train_steps_per_second': 9.048, 'total_flos': 560305631370240.0, 'train_loss': 3.266777502415815, 'epoch': 15.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-1.5820312 , -2.3789062 , -2.9003906 , ..., -1.6298828 ,
        -0.10827637,  6.1992188 ],
       [-2.4726562 , -3.0449219 , -0.8408203 , ..., -1.2783203 ,
        -1.1113281 ,  0.5961914 ],
       [-2.5996094 , -3.1015625 , -1.0810547 , ..., -1.2451172 ,
        -0.7583008 ,  0.671875  ],
       ...,
       [-0.2939453 , -0.6386719 , -2.1660156 , ..., -0.58251953,
         0.53027344,  2.6523438 ],
       [-0.9477539 , -0.7973633 ,  0.46142578, ..., -0.17089844,
        -0.99853516, -2.3554688 ],
       [-2.3203125 , -3.109375  , -2.1582031 , ..., -1.6914062 ,
        -0.47509766,  4.2851562 ]], dtype=float32), label_ids=array([27, 31, 24, 50, 80,  1, 35, 80, 58, 80, 55, 57, 80, 44, 52, 66, 80,
       52,  6, 21, 55, 36, 50, 37, 29, 23, 77, 35, 70, 74, 35, 30, 53,  4,
       77, 52,  9, 61, 22, 80, 71, 56, 19, 17, 52, 51, 72, 11, 72, 74, 79,
       72,  3,  8, 53, 80,  8,  6, 72, 36,  9, 19, 77, 36,  6, 16, 70,  8,
       24, 65, 76, 80, 35, 80, 7

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.328347,0.163121,0.005485,0.027290,0.009081
2,No log,4.208721,0.205674,0.014671,0.035088,0.018839
3,No log,3.989597,0.212766,0.039569,0.052986,0.040478
4,No log,3.688996,0.234043,0.043054,0.063353,0.044693
5,No log,3.449309,0.283688,0.045890,0.076023,0.047321
6,No log,3.181966,0.326241,0.059944,0.122807,0.074925
7,3.997200,2.998694,0.347518,0.054096,0.126026,0.070944
8,3.997200,2.886861,0.347518,0.046652,0.110775,0.062694
9,3.997200,2.799444,0.354610,0.068298,0.123700,0.078625
10,3.997200,2.741369,0.375887,0.080315,0.145320,0.093585


TrainOutput(global_step=960, training_loss=3.48919308980306, metrics={'train_runtime': 182.043, 'train_samples_per_second': 83.519, 'train_steps_per_second': 5.273, 'total_flos': 448244505096192.0, 'train_loss': 3.48919308980306, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-2.4902344 , -2.6523438 , -1.7148438 , ..., -1.1904297 ,
        -1.8837891 ,  5.7773438 ],
       [-1.3642578 , -1.1669922 , -0.1875    , ..., -0.58203125,
        -1.2724609 , -1.1357422 ],
       [-1.6962891 , -1.4658203 , -0.4572754 , ..., -0.6645508 ,
        -1.5498047 ,  0.26635742],
       ...,
       [-0.9663086 , -0.9794922 , -0.8017578 , ..., -0.6035156 ,
        -0.51660156,  2.5878906 ],
       [-0.10638428, -0.12493896,  0.2487793 , ...,  0.08642578,
         0.29370117, -2.6152344 ],
       [-2.1113281 , -2.2246094 , -1.1308594 , ..., -0.7548828 ,
        -1.75      ,  4.2148438 ]], dtype=float32), label_ids=array([27, 31, 24, 50, 80,  1, 35, 80, 58, 80, 55, 57, 80, 44, 52, 66, 80,
       52,  6, 21, 55, 36, 50, 37, 29, 23, 77, 35, 70, 74, 35, 30, 53,  4,
       77, 52,  9, 61, 22, 80, 71, 56, 19, 17, 52, 51, 72, 11, 72, 74, 79,
       72,  3,  8, 53, 80,  8,  6, 72, 36,  9, 19, 77, 36,  6, 16, 70,  8,
       24, 65, 76, 80, 35, 80, 7